## Jupyter notebook illustrating  commands required to use Python 2.7 modules

Copyright 2016 Ursa Analytics, Inc.
Licensed under the Apache License, Version 2.0 (the "License");
You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0



### Cell below loads the required modules and packages.  Code provided places all custom files in ./src.  3rd party dependencies assumed to be installed on system (Dockerfile provided illustratres one working computational environment).  

In [1]:
# load custom files provided with this repo under the Apache 2 license.
import MotBnB_Batch as MotionBlurFilter
import MotBnBwithNDClassKF_Batch
import findBerglundVersionOfMA1_Batch #this module builds off of Berglund's 2010 PRE parameterization (atypical MA1 formulation)

# imports below load standard 3rd party packages
%matplotlib inline 
#command above avoids using the pylab flag when launching ipython (always put magic command above as first arg to ipynb file)

#load some standard scientific computing and plotting modules (code tested using Python 2.7.10, Scipy 0.15.1, and Matplotlib 1.4.3)
import numpy as np
import h5py
import scipy.optimize as spo
import matplotlib.font_manager as font_manager
import matplotlib.pyplot as plt
####################################


def loadh5(floc,dsetnameIndex=0,printFile=False,printDsets=False): #load a numeric hdf5 file. "floc" is the path of ASCII file to read.
    """
    loadh5(floc,dsetnameIndex=0,printFile=False,printDsets=False): #load a numeric hdf5 file. "floc" is the path of ASCII file to read.

    """

    if floc[0]=='~':  #replace standard unix home shortcut with explicit path
        floc='/Users/calderoc' + floc[1:]
    f=h5py.File(floc,'r')
    dsetNames = f.keys()
    if printFile:
        print 'Read file: ' + floc
        print 'Pulling Dataset index', dsetnameIndex, 'Named: ' + dsetNames[ix]
    if printDsets:
        if printFile:
            print 'Displaying top level dsets in hdf5 file since printDset set to True'
            print 'Found N= ',len(f.keys()),'top level dsets (returning integer vs. any data since printDset=True)'
            for i in f.keys():
                print i
        return len(f.keys())
    else:
        dsetname=dsetNames[dsetnameIndex]
        g= f[dsetname]  #default name for h5import dataset when no arg given (so i use this in my scripts as well)
        g= np.array(g) #return numpy array (get shape with g.shape)
        f.close()
    return g 

def loadh5retdsetname(floc,dsetnameIndex=0,printFile=False,printDsets=False): #load a numeric hdf5 file. "floc" is the path of ASCII file to read.
    """
    loadh5retdsetname(floc,dsetnameIndex=0,printFile=False,printDsets=False): #load a numeric hdf5 file. "floc" is the path of ASCII file to read.

    """

    if floc[0]=='~':  #replace standard unix home shortcut with explicit path
        floc='/Users/calderoc' + floc[1:]
    f=h5py.File(floc,'r')
    dsetNames = f.keys()
    dsetname=dsetNames[0] #default to returning first name in list
    if printFile:
        print 'Read file: ' + floc
        print 'Pulling Dataset index', dsetnameIndex, 'Named: ' + dsetNames[ix]
    if printDsets:
        if printFile:
            print 'Displaying top level dsets in hdf5 file since printDset set to True'
            print 'Found N= ',len(f.keys()),'top level dsets (returning integer vs. any data since printDset=True)'
            for i in f.keys():
                print i
        return len(f.keys())
    else:
        dsetname=dsetNames[dsetnameIndex]
        g= f[dsetname]  #default name for h5import dataset when no arg given (so i use this in my scripts as well)
        g= np.array(g) #return numpy array (get shape with g.shape)
        f.close()
    return g,dsetname


def wouth5(floc,DATAMAT,dset='dataset0'): #write a simple hdf5 file into "floc";  assumes numpy array passed as DATAMAT.  handling other datatypes with h5py is fairly easy
    """
    def wouth5(floc,DATAMAT,dset='dataset0')
    write an hdf5 file dumping everything into generic name useful for playing with other programs "/dataset0."  

    floc: file location  (can use "~" in path) 
    DATAMAT: numpy matrix or vector

    """
    if floc[0]=='~':  #replace standard unix home shortcut with explicit path
                floc='/Users/calderoc' + floc[1:] #purdue login: calderoc / princeton: ccaldero; since grad school i use either calderoc or ccalderoN as unix logins (keep symlink to both home folders in *nix style systems)
    f=h5py.File(floc,'a')#default name for h5import dataset when no arg given (so i use this in my scripts as well)
    dset = f.create_dataset(dset, data=DATAMAT)
    f.close()
    return 0

##Now that required modules packages are loaded, set parameters for simulating "Blurred" OU trajectories.  Specific mixed continuous/discrete model:

\begin{align}
dr_t = & ({v}-{\kappa} r_t)dt + \sqrt{2 D}dB_t \\
\psi_{t_i} = & \frac{1}{t_E}\int_{t_{i}-t_E}^{t_i} r_s ds + \epsilon^{\mathrm{loc}}_{t_i}
\end{align}

###In above equations, parameter vector specifying model is: $\theta = (\kappa,D,\sigma_{\mathrm{loc}},v)$


###Statistically exact discretization of above for uniform time spacing  $\delta$  (non-uniform $\delta$ requires time dependent vectors and matrices below):

\begin{align}
r_{t_{i+1}} = & A + F r_{t_{i}} + \eta_{t_i} \\
\psi_{t_i} = & H_A + H_Fr_{t_{i-1}} + \epsilon^{\mathrm{loc}}_{t_i} + \epsilon^{\mathrm{blur}}_{t_i} \\
\epsilon^{\mathrm{loc}}_{t_i} + & \epsilon^{\mathrm{blur}}_{t_i} \sim  \mathcal{N}(0,R_i) \\
\eta_i \sim & \mathcal{N}(0,Q) \\
t_{i-1} \le & t_{i}-t_E \\
 C = & cov(\epsilon^{\mathrm{blur}}_{t_i},\eta_{t_{i-1}}) \ne 0
\end{align}


####Notes:  
##### 1) Kalman Filter (KF) and Motion Blur Filter (MBF) codes estimate $\sqrt{(2D)}$  as "thermal noise" parameter.  
##### 2) The expression $ t_{i-1} \le & t_{i}-t_E $ shows that non-uniform time spacing is possible.  Code provided can efficientle handle cases where "time lapse" is intentionally used (e.g. laser is on periodically) or in "blinking cases" (e.g. continuous illumination used, but fluor enters dark state causing particle to be missed in some frames).

In [2]:
#set parameters determining state dynamics.  everything in this block can by changed in batch.  
# keep derived quantities in cell below read in.
# kappa = 1. # units: [1/s] %code currently setup to set kappa to achieve desired "corral radius" defined by L.
v = 0. #units [microns/s]
D = 1. #units [microns^2/s] %use this to show convergence
L= 500/1000. #[radius or box width in microns]
#parmeters determining trajectory length, uniform spacing in time, and number of trajectories to analyze 
offset=0. #if no localization accuracy info, probably not a bad idea to have zero offset and put in a high constant initial guess for noise
locIG = 40./1000.

ds=1 #"down sampling" integer parameter.  
delta = 10./1000. #spacing between "image observations" under "continuous illumination" sampling
#for time lapse model used, assumes time between two image frames is ds*delta.  code can be modified to handle "more exotic" situations
exposureTime=delta #this is a fixed experimental control parameter.  set input parameter needed 
#(exposureTime=delta may look redudant based on comment, but aux varialbe is used to identify "gaps" induced by blinking)
deltaNET = exposureTime*ds #define as net time of frame (not exposure time) for time lapse tests.  
#Heuristic: if t_E/deltaNET is small, MBF correction will be small (e.g. analyzing as standard KF will likely introduce little approximation error)  
#for non-uniform time spacing, need to specify a time vector (code illustrates how to set this up)






In [3]:
kappa = 2*D*6/(L**2)
print 'target L [nm]: ', L*1000
print 'kappa yielding target for given D',kappa

twoDsqrt = np.sqrt(D*2.) #filter codes provided (KF and MBF) estimate square root of 2D directly.
trueRatio = D/kappa
print 'D/kappa true:',trueRatio
print 'L^2/6',L**2/6.



pars=(-2,0,0,-2,twoDsqrt,0,twoDsqrt, 30./1000.,30./1000.,0,0)
print 'baseline pars',pars

target L [nm]:  500.0
kappa yielding target for given D 48.0
D/kappa true: 0.0208333333333
L^2/6 0.0416666666667
baseline pars (-2, 0, 0, -2, 1.4142135623730951, 0, 1.4142135623730951, 0.03, 0.03, 0, 0)


In [4]:
import glob2

datadir='./cp2019_401/**/*.h5'

# datadir='./ManualEGs/**/*.h5' # These are the control cells, 



# datadir='TA (Sec61)/**/*.csv' # These are cells pharmacologically treated that should slow down luminal motion (at least at the ensemble level as demonstrated by FRAP)
srcFiles = glob2.glob(datadir)
#with this approach, output cannot exist, but h5py and formulation crashes if files exists when trying to duplicate dataset, so just be sure to move or erase output files
#containint "_BlurRes","_KFRes",etc
# print srcFiles[0]
print srcFiles,len(srcFiles)


['./cp2019_401/cp2019_401.h5'] 1


In [ ]:
np.random.seed(0) #set seed to allow reproducible data.  
myoptmeth='nelder-mead' #set optimization for MLE search (recommended to use nelder-mead due to highly nonlinear nature of MLE)


pltFlag = False

for fname in srcFiles :
# for fname in srcFiles[0:1]: #test case (don't just give single values since strings are iterable...)
    Nbatch = loadh5retdsetname(fname,printDsets=True)
    print 'Found',Nbatch, 'trajectories in ',fname
    
    #inset code specific read here.  make a routine for pulling batches
    for ni in range(Nbatch):
#     for ni in range(2):
        
        batchY = [] #reset data passed to arg.  if desired to save, simple mod (probably best to just set random number seed)
        batchNoiseMag = []
        batchTdata = []
        print '\n','^'*100 
        print 'Batch ', ni + 1 , ' of ', Nbatch, '&'*50
        print '^'*100 ,'\n\n'
        
        data,dsetname = loadh5retdsetname(fname,dsetnameIndex=ni)
        batchTdata = [data[:,1]] #need lists as input
        batchXYdata = data[:,2:4] #added meta info in form of dz to last column.  be explicit with frame,time,x,y,z
        
        
        if pltFlag:
            print fname
            plt.plot(batchXYdata[0,:],batchXYdata[1,:],'-o')
            plt.show()
            plt.plot(batchXYdata[0,:],'->')
            plt.plot(batchXYdata[1,:],'-x')
            plt.show()
        
        #setup some lists to store results o single trajectory in file 
        res1=[];res2=[];res3=[];res1b=[];res2Bias=[];res3Bias=[];
        
#         for traji in batchXYdata:
        for fauxvar in range(1):
            
            traji = batchXYdata
            print 'Demeaning data:'
            traji = traji - np.mean(traji,axis=0)
            
            batchY=[traji]
        
        #     BlurF = MotionBlurFilter.ModifiedKalmanFilter1DwithCrossCorr(batchY,batchTdata,StaticErrorEstSeq= batchNoiseMag)
#             batchNoiseMag = [np.ones(batchTdata[0].shape)*locIG]
#             BlurF = MotionBlurFilter.ModifiedKalmanFilter1DTimeLapse(batchY,batchTdata,exposureTime,StaticErrorEstSeq= batchNoiseMag)
            KF = MotBnBwithNDClassKF_Batch.KalmanFilterND(batchY,batchTdata)
            
#             MA = findBerglundVersionOfMA1_Batch.CostFuncMA1Diff_Batch(batchY,deltaNET,blurCoef=1./6.)
#             MAb = findBerglundVersionOfMA1_Batch.CostFuncMA1Diff_Batch(batchY,deltaNET,blurCoef=0)
        #     BlurF = MotionBlurFilter.ModifiedKalmanFilter1DTimeLapse(batchY,batchTdata,exposureTime)
#             KF = MotionBlurFilter.ClassicKalmanFilter(batchY,batchTdata)
        #     MA = findBerglundVersionOfMA1_Batch.CostFuncMA1Diff_Batch(batchY,deltaNET,blurCoef=1./6.)
        #     MAb = findBerglundVersionOfMA1_Batch.CostFuncMA1Diff_Batch(batchY,deltaNET,blurCoef=0)
            
            parsIG = [3.*i/4. for i in pars] #kick off simulations with a biased estimate of truth (for long trajectories IC, less relevant, but for small sample sizes, need reasonable IC since local minima can be problematics)
            
        #repeat optimization to ensure minimal IG dependence (important for short traj)
            Nrepeat = 3
            maxHist = [np.inf,np.inf,np.inf,np.inf] #set inf as initial value
            maxRes= [[],[],[],[]] #give bogus IGs
            
            for nrpt in range(Nrepeat):
                parsIG = [i+i*.2*np.random.normal(1) for i in pars]
                
#                 resMAtmp = spo.minimize(MA.evalCostFuncVel_Batch,[parsIG[1]/np.sqrt(2),parsIG[2],parsIG[-1]], method=myoptmeth)
#                 resMAbtmp = spo.minimize(MAb.evalCostFuncVel_Batch,[parsIG[1]/np.sqrt(2),parsIG[2],parsIG[-1]], method=myoptmeth)
#                 resBlurtmp = spo.minimize(BlurF.evalCostFunc,parsIG, method=myoptmeth)            
                resKFtmp = spo.minimize(KF.evalCostFunc,parsIG, method=myoptmeth)
                resBlurtmp=resKFtmp;resMAtmp=resKFtmp;resMAbtmp=resKFtmp #use place holders to avoid making changes below
                #store the optimization output structure
                tmpVec = [resMAtmp,resMAbtmp,resBlurtmp,resKFtmp]
                
                #cycle through and update based on max vals obtained
                for ix,resi in enumerate(tmpVec):
                    fval = resi.fun
                    if fval<maxHist[ix]:
                        maxRes[ix]=resi
                        maxHist[ix]=fval
                        

                resKF=maxRes[3]
                
            
            

            print 'Classic 2D KF results:',resKF
            print '$'*100
            res3.append(resKF.x)

    
        #store results in hdf5 file
        fbase = fname[:-3]
#         wouth5(fbase+'_MARes'+'.h5',res1,dset=dsetname)
#         wouth5(fbase+'_MAResBlurCoefRes'+'.h5' ,res1b,dset=dsetname)
#         wouth5(fbase+'_BlurRes'+'.h5' ,res2,dset=dsetname)
        wouth5(fbase+'_KF2DRes'+'.h5',res3,dset=dsetname)
        
    


Found 1054 trajectories in  ./cp2019_401/cp2019_401.h5

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Batch  1  of  1054 &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ 


Demeaning data:
Classic 2D KF results:  final_simplex: (array([[-5.40080087e+00, -7.32892657e-01,  1.32459652e-01,
        -2.56200396e-01,  1.56073728e+00, -2.08408394e-01,
         1.82600185e+00,  3.89522165e-02,  4.22167065e-03,
        -1.04375986e-01,  2.49609095e-01],
       [-5.33321416e+00, -7.19841970e-01,  1.29993644e-01,
        -2.52490722e-01,  1.57234246e+00, -2.04800939e-01,
         1.83066118e+00,  3.74858323e-02,  5.88070543e-03,
        -1.02404615e-01,  2.45282601e-01],
       [-5.41428657e+00, -7.39876785e-01,  1.33701914e-01,
        -2.58379492e-01,  1.57596505e+00, -2.10421600e-01,
         1.82020502e+00,  3.82145692e-02,  4.27308350e-